In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from pathlib import Path

In [ ]:
from watch_recognition.data_preprocessing import load_keypoints_data_as_kp
ROOT_DIR = ".."
X, y, filenames = load_keypoints_data_as_kp(
    Path("%s/download_data/keypoints/train" % ROOT_DIR),
)
X.shape, y.shape

In [ ]:
X_val, y_val, filenames = load_keypoints_data_as_kp(
    Path("%s/download_data/keypoints/validation" % ROOT_DIR),
)
X_val.shape, y_val.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
example_idx = 11
example_X = X[example_idx]
example_y = y[example_idx]
plt.imshow(example_X)

In [ ]:
center = example_y[0,:2]
top = example_y[1,:2] - center
center = center - center

In [ ]:
top = top / 224

In [ ]:
reference_vector = np.array([0,-0.5])

In [ ]:
angle = (
        np.rad2deg(np.arctan2(*top) - np.arctan2(*reference_vector))
    )


In [ ]:
angle

In [ ]:
top

In [ ]:
plt.imshow(example_X)
plt.scatter(*(center + 112))
plt.scatter(*((top * 224) + 112))
plt.scatter(*((reference_vector * 224) + 112))

In [ ]:
from skimage.transform import rotate

In [ ]:
rotated_image = (rotate(example_X, -angle) * 255).astype('uint8')
plt.imshow(rotated_image)

In [ ]:

import tensorflow as tf
model_path = "./models/keypoint/efficientnetb0/run_1634142458.893467.h5"
loaded_model = tf.keras.models.load_model(model_path, compile=False)

In [ ]:
from watch_recognition.reports import run_on_image_debug
run_on_image_debug(loaded_model, example_X)

In [ ]:
plt.imshow(rotated_image)

In [ ]:
run_on_image_debug(loaded_model, rotated_image)

In [ ]:
from watch_recognition.data_preprocessing import keypoints_to_angle, binarize

In [ ]:
from itertools import product

In [ ]:
xs = np.linspace(0,1, 20)
ys = np.linspace(0,1, 20)
center = np.array([0.5,0.5])
angles = []
for x_i, y_i in product(xs, ys):
    top = np.array([x_i, y_i])
    angle = keypoints_to_angle(center, top)
    angles.append(angle)
plt.hist(angles, bins=36)

In [ ]:
from distinctipy import distinctipy

# number of colours to generate
bin_size = 90
N = 360 // bin_size

# generate N visually distinct colours
colors = distinctipy.get_colors(N)

# display the colours
distinctipy.color_swatch(colors)

xs = np.linspace(0,1, 20)
ys = np.linspace(0,1, 20)
center = np.array([0.5,0.5])
angles = []
for x_i, y_i in product(xs, ys):
    top = np.array([x_i, y_i])
    angle = keypoints_to_angle(center, top)
    angles.append(angle)
    cls = binarize(angle, bin_size)
    plt.scatter(x_i,y_i, color=colors[cls])

In [ ]:
from tensorflow.python.keras.utils.np_utils import to_categorical

In [ ]:
def encode_targets(y, bin_size=10):
    y_targets = []
    for y_ in y:
        center =y_[0,:2]
        top = y_[1,:2]
        angle = keypoints_to_angle(center, top)
        angle = binarize(angle, bin_size)
        angle_one_hot = to_categorical(angle, num_classes=360 // bin_size)
        y_targets.append(angle_one_hot)
    return np.array(y_targets).astype('float32')

In [ ]:
y_targets = encode_targets(y, bin_size=bin_size)

In [ ]:
plt.hist(y_targets.argmax(axis=1))
plt.show()

In [ ]:
for kp, cls in zip(y, y_targets.argmax(axis=1)):
    top = kp[1,:2]
    plt.scatter(*top, color=colors[cls])
plt.gca().invert_yaxis()

In [ ]:
import pandas as pd

In [ ]:
pd.value_counts(pd.Series(y_targets.argmax(axis=1)), normalize=True)

In [ ]:
y_targets_val = encode_targets(y_val, bin_size=bin_size)

In [ ]:
plt.hist(y_targets_val.argmax(axis=1))

In [ ]:
import ipyplot

In [ ]:
ipyplot.plot_class_tabs(X, y_targets.argmax(axis=1))

In [ ]:
ipyplot.plot_class_tabs(X_val, y_targets_val.argmax(axis=1))

In [ ]:
for kp, cls in zip(y_val, y_targets_val.argmax(axis=1)):
    top = kp[1,:2]
    plt.scatter(*top, color=colors[cls])
plt.gca().invert_yaxis()

In [ ]:
backbone = tf.keras.applications.EfficientNetB0(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)

outputs = [
    backbone.get_layer(layer_name).output for layer_name in ["block7a_project_conv"]
]
base_model = tf.keras.Model(inputs=[backbone.inputs], outputs=outputs)
inputs = tf.keras.Input(
    shape=(224, 224, 3),
)
x = base_model(inputs)
options = {
    "kernel_initializer": tf.keras.initializers.RandomNormal(
        mean=0.0, stddev=0.01, seed=None
    ),
    "bias_initializer": "zeros",
}

for i in range(3):
    x = tf.keras.layers.Conv2D(filters=160, kernel_size=3, activation=None)(x)
    x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
    x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)

x = tf.keras.layers.Conv2D(
    filters=360 // bin_size, kernel_size=1, activation="softmax"
)(x)
output = tf.keras.layers.Flatten()(x)

model = tf.keras.models.Model(inputs=inputs, outputs=output)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=[
        "acc",
    ],
    optimizer=tf.keras.optimizers.Adam(3e-4)
)

In [ ]:
from watch_recognition.augmentations import set_shapes, process_kp_data, encode_keypoints_to_angle, get_augmented_watch_angle_dataset

In [ ]:
from functools import partial

In [ ]:
dataset = get_augmented_watch_angle_dataset(X, y, bin_size=bin_size)

In [ ]:
angles = []
for batch in dataset.take(32*17):
    angles.extend(batch[1].numpy().argmax(axis=1).flatten())
plt.hist(angles, bins=8)

In [ ]:
def view_image(ds):
    image, angles = next(iter(ds))  # extract 1 batch from the dataset
    image = image.numpy()
    angles = angles.numpy().argmax(axis=1)
    fig, axarr = plt.subplots(5, 2, figsize=(15, 15))
    for i in range(5):
        ax = axarr[i]
        img = image[i]
        ax_idx = 0
        ax[ax_idx].imshow(img.astype("uint8"))
        ax[ax_idx].set_xticks([])
        ax[ax_idx].set_yticks([])
        ax[ax_idx].set_title("Image")

        ax_idx += 1
        rotated_image = rotate(img.astype('float32'), -angles[i]*45)
        ax[ax_idx].imshow(rotated_image.astype('uint8'))
        ax[ax_idx].set_xticks([])
        ax[ax_idx].set_yticks([])
        ax[ax_idx].set_title("Corrected angle")
view_image(dataset)

In [ ]:
EPOCHS = 100
history = model.fit(
    dataset,
    epochs=EPOCHS,
    validation_data=(X_val, y_targets_val),
#     callbacks = [tf.keras.callbacks.ReduceLROnPlateau()]
)

In [ ]:
history = model.history

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
# plt.yscale('log')

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

In [ ]:
for i in range(10):
    example = X[i:i+1]
    pred = model.predict(example).argmax(axis=1) *bin_size
    print(pred, y_targets[i:i+1]*bin_size)
    rotated_image = (rotate(example[0], -pred[0]) * 255).astype('uint8')
    plt.imshow(example[0])
    plt.show()
    plt.imshow(rotated_image)
    plt.show()

In [ ]:
for i in range(10):
    example = X_val[i:i+1]
    pred = model.predict(example).argmax(axis=1) *bin_size
    print(y_targets_val[i:i+1]*bin_size, pred)
    rotated_image = (rotate(example[0], -pred[0]) * 255).astype('uint8')
    plt.imshow(example[0])
    plt.show()
    plt.imshow(rotated_image)
    plt.show()

In [ ]:
bad_train_predictions = X[y_targets.argmax(axis=1) != model.predict(X).argmax(axis=1)]

In [ ]:
1 - len(bad_train_predictions) / len(X)

In [ ]:
bad_train_predictions.shape

In [ ]:
bad_val_predictions = X_val[y_targets_val.argmax(axis=1) != model.predict(X_val).argmax(axis=1)]

In [ ]:
bad_val_predictions.shape

In [ ]:
1 - len(bad_val_predictions) / len(X_val)

In [ ]:
len(bad_val_predictions)

In [ ]:
model.save("rotation-model-cls-4-fc-4.h5")

In [ ]:
path = '../example_data/Zrzut ekranu 2021-08-25 o 22.24.24.jpg'
test_image = tf.keras.preprocessing.image.load_img(
        path, "rgb", target_size=(224,224), interpolation="bicubic",
    )
test_image_np = tf.keras.preprocessing.image.img_to_array(test_image)


In [ ]:
for a in np.linspace(0, 360, 10):
    fig, axarr = plt.subplots(1, 2, figsize=(15,15))
    rotated_image = np.expand_dims((rotate(test_image_np.astype('float32'), a)).astype('uint8'), axis=0)
    pred = model.predict(rotated_image).argmax(axis=1) * bin_size
    print(a,pred[0])
    axarr[0].imshow(rotated_image[0])
    rotated_image = (rotate(rotated_image[0].astype('float32'), -pred[0])).astype('uint8')
    axarr[1].imshow(rotated_image)
    run_on_image_debug(loaded_model, rotated_image)
    
    plt.show()
    